In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
Y_train = np.loadtxt('data/train.txt').astype(int)
Y_test = np.loadtxt('data/test.txt').astype(int)
ratings = Y_train[:, 2:]
mean_rating = np.mean(ratings)

In [3]:
# Solution set for CS 155 Set 6, 2016/2017
# Authors: Fabian Boemer, Sid Murching, Suraj Nair

import numpy as np

def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return (1-reg*eta)*Ui + eta * Vj * (Yij - np.dot(Ui,Vj))     

def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return (1-reg*eta)*Vj + eta * Ui * (Yij - np.dot(Ui,Vj))

def get_err(U, V, Y, b1, b2, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    # Compute mean squared error on each data point in Y; include
    # regularization penalty in error calculations.
    # We first compute the total squared squared error
    err = 0.0
    for (i,j,Yij) in Y:
        err += 0.5 *(Yij - ((np.dot(U[i-1], V[:,j-1])) + mean_rating + b1[i-1] + b2[j-1]))**2
    # Add error penalty due to regularization if regularization
    # parameter is nonzero
    if reg != 0:
        U_frobenius_norm = np.linalg.norm(U, ord='fro')
        V_frobenius_norm = np.linalg.norm(V, ord='fro')
        err += 0.5 * reg * (U_frobenius_norm ** 2)
        err += 0.5 * reg * (V_frobenius_norm ** 2)
    # Return the mean of the regularized error
    return err / float(len(Y))

def get_single_err(U, V, i, j, Yij, b1, b2, reg=0.0):
    prediction = mean_rating + b1[i-1] + b2[j-1] + (np.dot(U[i-1], V[:,j-1]))
    error = Yij - prediction
    return error

def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    # Initialize U, V  
    U = np.random.random((M,K)) - 0.5
    V = np.random.random((K,N)) - 0.5
    bias_u = np.zeros(M)
    bias_v = np.zeros(N)
    size = Y.shape[0]
    delta = None
    indices = np.arange(size)    
    for epoch in range(max_epochs):
        # Run an epoch of SGD
        before_E_in = get_err(U, V, Y, bias_u, bias_v, reg)
        np.random.shuffle(indices)
        for ind in indices:
            (i,j, Yij) = Y[ind]
            # Update U[i], V[j]
            error = get_single_err(U, V, i, j, Yij, bias_u, bias_v, reg)
            bias_u[i-1] += eta * (error - reg * bias_u[i-1])
            bias_v[j-1] += eta * (error - reg * bias_v[j-1])
            U[i-1] = grad_U(U[i-1], Yij, V[:,j-1], reg, eta)
            V[:,j-1] = grad_V(V[:,j-1], Yij, U[i-1], reg, eta);
        # At end of epoch, print E_in
        E_in = get_err(U, V, Y, bias_u, bias_v, reg)
        print("Epoch %s, E_in (regularized MSE): %s"%(epoch + 1, E_in))

        # Compute change in E_in for first epoch
        if epoch == 0:
            delta = before_E_in - E_in

        # If E_in doesn't decrease by some fraction <eps>
        # of the initial decrease in E_in, stop early 
        elif before_E_in - E_in < eps * delta:
            break
    return (U, V, get_err(U, V, Y, bias_u, bias_v, reg), bias_u, bias_v)

In [5]:
M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
print("Factorizing with ", M, " users, ", N, " movies.")
K = 20 
reg = 0.1
eps = 0.1
eta = 0.03 # learning rate

Factorizing with  943  users,  1682  movies.


In [8]:
print("Training model with M = %s, N = %s, k = %s, eta = %s, reg = %s"%(M, N, K, eta, reg))
U,V, e_in, bias_u, bias_v = train_model(M, N, K, eta, reg, Y_train, eps)
eout = get_err(U, V, Y_test, bias_u, bias_v)
print("Ein: ", e_in, " Eout: ", eout)

Training model with M = 943, N = 1682, k = 20, eta = 0.03, reg = 0.1
Epoch 1, E_in (regularized MSE): 0.654179081812
Epoch 2, E_in (regularized MSE): 0.494047769858
Epoch 3, E_in (regularized MSE): 0.453553424338
Epoch 4, E_in (regularized MSE): 0.42560947365
Epoch 5, E_in (regularized MSE): 0.40959946625
Epoch 6, E_in (regularized MSE): 0.395294534148
Epoch 7, E_in (regularized MSE): 0.389071415895
Epoch 8, E_in (regularized MSE): 0.379755148686
Epoch 9, E_in (regularized MSE): 0.369758697352
Epoch 10, E_in (regularized MSE): 0.370075161504
Ein:  0.370075161504  Eout:  0.472148622408


In [42]:
A, s, B = np.linalg.svd(V, full_matrices=False)
A_2 = A[:, :2]
A_2_transpose = np.transpose(A_2)
print(A_2_transpose.shape)
U_t = np.dot(A_2_transpose, U.transpose())
print(U_t.shape)
V_t = np.dot(A_2_transpose, V)
print(V_t.shape)

(2, 20)
(2, 943)
(2, 1682)


In [43]:
for i in range(len(U_t)):
    U_t[i] -= np.mean(U_t[i])
    V_t[i] -= np.mean(V_t[i])
    U_t[i] /= np.std(U_t[i])
    V_t[i] /= np.std(V_t[i])

In [51]:
U_vis = np.transpose(U_t)
V_vis = np.transpose(V_t)
print(U_vis)

[[-0.77544502 -1.35707864]
 [-0.98784665 -0.66430133]
 [ 1.77880392 -0.37152816]
 ..., 
 [-1.17367395  0.02589783]
 [-1.52200354  0.71018566]
 [-0.36736303  0.42419429]]
